#Montar Disco

In [1]:
from google.colab import drive
drive.mount('/content/MyDrive')


Mounted at /content/MyDrive


#Importar Librerias

In [2]:
# importar librerias 
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import TensorBoard
import keras
import numpy as np
import  cv2

#Datos

In [21]:
# ruta de la carpeta con la imagenes procesadas
folder = '/content/MyDrive/MyDrive/ProyectoIA_U3/New_data'
# tamaño de batch o lote 
batch_size = 60
#altura de  imagen 
img_height = 192
# ancho de imagen 
img_width = 192

In [22]:
# data set de entrenamiento 
train_ds = tf.keras.utils.image_dataset_from_directory(
  folder, #ruta de las carpetas 
  validation_split=0.1, # porcetanje de datos para  validacion
  subset="training", # especificamos que es el dataset de entrenamiento 
  color_mode ='rgb', # modo de imagen rgb ==> 3 canales
  seed=222, # semilla para  el den aleatorio de  imagenes 
  label_mode = 'int', # modo la etiqueta 
  image_size=(img_height, img_width), # alto  ancho de imagen 
  batch_size=batch_size # batch 
  ) 

Found 4077 files belonging to 26 classes.
Using 3670 files for training.


In [23]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  folder, #ruta de las carpetas 
  validation_split=0.1,# porcetanje de datos para  validacion
  subset="validation",# especificamos que es el dataset de validacion
  color_mode ='rgb',# modo de imagen rgb ==> 3 canales
  seed=222,# semilla para  el den aleatorio de  imagenes 
  label_mode = 'int',# modo la etiqueta 
  image_size=(img_height, img_width),# alto  ancho de imagen 
  batch_size=batch_size)# batch 

Found 4077 files belonging to 26 classes.
Using 407 files for validation.


In [24]:
class_names = train_ds.class_names
print(len(class_names))
class_names = val_ds.class_names
print((class_names))

26
['Ariel Chabla', 'ArroboMercy', 'BarraganSteven', 'CevallosJoan', 'EnriquezSelena', 'GenesisHeredia', 'GoyesAnthony', 'HectorCedeno', 'Jhon Zambrano', 'Jordan Espinosa', 'Jorge Borrero', 'JoseRuiz', 'LucioCarlos', 'MasacheFernando', 'MelanyLopez', 'MosqueraLucy', 'Nataly Acosta', 'OlallaLuis', 'ParragaMariaJose', 'PauteKevin', 'Raymond Davila', 'RivasSelena', 'SalazarJohana', 'Solano Wilmer', 'SolorzanoBryan', 'VinicioBorja']


In [25]:
# muestra o imprime la forma de los tensores de la imagen y etiqueta
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(60, 192, 192, 3)
(60,)


In [27]:
# muestra o imprime la forma de los tensores de la imagen y etiqueta
images = []
labels = []
for image_batch, labels_batch in train_ds:
  images.append(image_batch)
  images.append(labels_batch)
  break

In [ ]:
# mostrar los valore de los  pixels en el tensor de images
print(images)

#Normalizacion

In [54]:

#capa de normalizacion para pasar los valores de los pixles de 0 a 255 y nomralizarolos entre 0 y 1
normalization_layer = tf.keras.layers.Rescaling(1./255)
#aplico la normalizacion al dat set de entrenamiento 
Ntrain_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
#aplico la normalizacion al dat set de validacion
Nval_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

image_batch, labels_batch = next(iter(Nval_ds))
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image))

# optimizacion para guardar  valores en cache 
AUTOTUNE = tf.data.AUTOTUNE
Ntrain_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
Nval_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

0.0 1.0


#AlexNet

In [36]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [35]:
import datetime
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

In [46]:
# ejemplo creacion rapida 
modelo1 = tf.keras.models.Sequential()
modelo1.add(tf.keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(192,192,3)))
modelo1.add(tf.keras.layers.Dense(100,activation='relu'))
modelo1.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 46, 46, 96)        34944     
                                                                 
 dense_2 (Dense)             (None, 46, 46, 100)       9700      
                                                                 
Total params: 44,644
Trainable params: 44,644
Non-trainable params: 0
_________________________________________________________________


In [41]:
modelo1.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 96)        34944     
                                                                 
Total params: 34,944
Trainable params: 34,944
Non-trainable params: 0
_________________________________________________________________


##Alexnet

In [47]:
Alexnet = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(192,192,3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    tf.keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2048, activation='relu'),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(26, activation='softmax')
])

Alexnet.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 46, 46, 96)        34944     
                                                                 
 batch_normalization (BatchN  (None, 46, 46, 96)       384       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 22, 22, 96)       0         
 )                                                               
                                                                 
 conv2d_7 (Conv2D)           (None, 22, 22, 256)       614656    
                                                                 
 batch_normalization_1 (Batc  (None, 22, 22, 256)      1024      
 hNormalization)                                                 
                                                      

##Optimizador 


In [ ]:

#learning rate
lr_rate = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0003,
    decay_steps=1000,
    decay_rate=0.9)

#ADAM
optAdam = tf.keras.optimizers.Adam(
    learning_rate=lr_rate)
#SGD
optSgd=tf.keras.optimizers.SGD(
    learning_rate=lr_rate,
    momentum=0.9,
    nesterov=False,
    name='SGD')

##Compiler

In [48]:
Alexnet.compile(
  optimizer='Adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])


##Callbacks

In [52]:
alexnetlog = TensorBoard(log_dir='/content/model')

earlingstop = tf.keras.callbacks.EarlyStopping(
    monitor='accuracy',
    patience=10,
    verbose=1,
    mode='auto',
    baseline=None,
    restore_best_weights=False
)

Saves = '/content/model/Alexnet34.h5'
#cheppoints
Checkp = tf.keras.callbacks.ModelCheckpoint(
    Saves,
    monitor='accuracy',
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch',
    options=None,
    initial_value_threshold=None
)



##Entrenamiento

In [55]:
hist = Alexnet.fit(
  Ntrain_ds,
  validation_data=Nval_ds,
  epochs=100,
  callbacks = [alexnetlog,earlingstop,Checkp]
)

Epoch 1/100


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


62/62 [==============================] - 687s 11s/step - loss: 4.4151 - accuracy: 0.0766 - val_loss: 21.7057 - val_accuracy: 0.0590
Epoch 2/100
62/62 [==============================] - 6s 97ms/step - loss: 2.8072 - accuracy: 0.1706 - val_loss: 7.0134 - val_accuracy: 0.0663
Epoch 3/100
62/62 [==============================] - 6s 95ms/step - loss: 2.4269 - accuracy: 0.2515 - val_loss: 3.7673 - val_accuracy: 0.1007
Epoch 4/100
62/62 [==============================] - 6s 96ms/step - loss: 2.0731 - accuracy: 0.3490 - val_loss: 3.6755 - val_accuracy: 0.1302
Epoch 5/100
62/62 [==============================] - 6s 95ms/step - loss: 1.7758 - accuracy: 0.4240 - val_loss: 3.6981 - val_accuracy: 0.1572
Epoch 6/100
62/62 [==============================] - 6s 95ms/step - loss: 1.5865 - accuracy: 0.5011 - val_loss: 3.8170 - val_accuracy: 0.2408
Epoch 7/100
62/62 [==============================] - 6s 95ms/step - loss: 1.4329 - accuracy: 0.5422 - val_loss: 2.8000 - val_accuracy: 0.3489
Epoch 8/100
62/6

KeyboardInterrupt: ignored

#Grid Search


In [56]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from matplotlib import pyplot as plt
import sys
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy

from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

from matplotlib.rcsetup import ValidateInterval
from tensorflow.python.ops.variables import VariableAggregationV2
import tensorflow as tf

In [57]:
datagen = ImageDataGenerator(rescale=1.0/255.0)
dataset = datagen.flow_from_directory(folder, class_mode='categorical', batch_size= 50)

X_image, Y_labels = dataset.next()

Found 4077 images belonging to 26 classes.


In [58]:
def create_modelA():
  Alexnet = tf.keras.models.Sequential([

          tf.keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(256,256,3)),
          tf.keras.layers.BatchNormalization(),
          tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
          tf.keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
          tf.keras.layers.BatchNormalization(),
          tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
          tf.keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
          tf.keras.layers.BatchNormalization(),
          tf.keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
          tf.keras.layers.BatchNormalization(),
          tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
          tf.keras.layers.BatchNormalization(),
          tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(4096, activation='relu'),
          tf.keras.layers.Dropout(0.3),
          tf.keras.layers.Dense(4096, activation='relu'),
          tf.keras.layers.Dropout(0.5),
          tf.keras.layers.Dense(2048, activation='relu'),
          tf.keras.layers.Dense(1024, activation='relu'),
          tf.keras.layers.Dropout(0.5),
          tf.keras.layers.Dense(26, activation='softmax')
      ])
        # compile model
        #opt = SGD(lr=0.001, momentum=0.9)
  opt = tf.keras.optimizers.SGD( learning_rate=0.01,    momentum=0.9)
  Alexnet .compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return Alexnet
seed = 7
numpy.random.seed(seed)


In [59]:
model = KerasClassifier(build_fn=create_modelA, verbose=0)

batch_size = [32,80, 100]
epochs = [ 10, 50, 100]

param_grid = dict(batch_size=batch_size, epochs=epochs)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1,verbose=0, cv=3)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


In [60]:
grid_result = grid.fit(X_image, Y_labels)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.139706 using {'batch_size': 80, 'epochs': 50}
0.000000 (0.000000) with: {'batch_size': 32, 'epochs': 10}
0.020833 (0.029463) with: {'batch_size': 32, 'epochs': 50}
0.020833 (0.029463) with: {'batch_size': 32, 'epochs': 100}
0.039216 (0.055459) with: {'batch_size': 80, 'epochs': 10}
0.139706 (0.026169) with: {'batch_size': 80, 'epochs': 50}
0.120098 (0.048154) with: {'batch_size': 80, 'epochs': 100}
0.139706 (0.026169) with: {'batch_size': 100, 'epochs': 10}
0.040441 (0.028636) with: {'batch_size': 100, 'epochs': 50}
0.078431 (0.073366) with: {'batch_size': 100, 'epochs': 100}
